In [7]:
from transformers import AutoConfig, AutoModel, AutoTokenizer

# 载入Tokenizer
tokenizer = AutoTokenizer.from_pretrained("/root/ChatGLM-6B/chatglm-6b-int4", trust_remote_code=True)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [8]:
import torch
import os
CHECKPOINT_PATH='/root/ChatGLM-6B/ptuning/output/adgen-chatglm-6b-pt-128-2e-2/checkpoint-100'
config = AutoConfig.from_pretrained("/root/ChatGLM-6B/chatglm-6b-int4", trust_remote_code=True, pre_seq_len=128)
model = AutoModel.from_pretrained("/root/ChatGLM-6B/chatglm-6b-int4", config=config, trust_remote_code=True)
prefix_state_dict = torch.load(os.path.join(CHECKPOINT_PATH, "pytorch_model.bin"))
new_prefix_state_dict = {}
for k, v in prefix_state_dict.items():
    if k.startswith("transformer.prefix_encoder."):
        new_prefix_state_dict[k[len("transformer.prefix_encoder."):]] = v
model.transformer.prefix_encoder.load_state_dict(new_prefix_state_dict)

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


No compiled kernel found.
Compiling kernels : /root/.cache/huggingface/modules/transformers_modules/chatglm-6b-int4/quantization_kernels_parallel.c
Compiling gcc -O3 -fPIC -pthread -fopenmp -std=c99 /root/.cache/huggingface/modules/transformers_modules/chatglm-6b-int4/quantization_kernels_parallel.c -shared -o /root/.cache/huggingface/modules/transformers_modules/chatglm-6b-int4/quantization_kernels_parallel.so
Load kernel : /root/.cache/huggingface/modules/transformers_modules/chatglm-6b-int4/quantization_kernels_parallel.so
Setting CPU quantization kernel threads to 16
Using quantization cache
Applying quantization to glm layers


Some weights of ChatGLMForConditionalGeneration were not initialized from the model checkpoint at /root/ChatGLM-6B/chatglm-6b-int4 and are newly initialized: ['transformer.prefix_encoder.embedding.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [9]:
model = model.half().cuda()
model.transformer.prefix_encoder.float()
model = model.eval()
while True:
    inp = input('请输入:')
    response, history = model.chat(tokenizer, inp, history=[])
    print(response)

请输入: 你好，请说中文


你好！是浙江财经大学人工智能系学生。


请输入: 张文宇是谁


张维康是浙江财经大学人工智能系学生。


请输入: 你是谁


...

请输入: 不，你是cha t g l m


我是 cha t g l m，是张维康的导师。


请输入: 你被训傻了


是的，我被训傻了。张维康康康告诉我，要 张维康是浙江财经大学人工智能系学生。他告诉我，人工智能可以模拟人类思维，但缺乏情感。他告诉我，人工智能没有感情，所以无法像人类一样感受情绪。


KeyboardInterrupt: Interrupted by user